In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

# ===========================
# CONFIG
# ===========================
file_name = "stock_metrics_121625.csv"   # Update as needed

CSV_PATH = f"/Users/adamtaylor/Documents/Personal/Finance/StockDB/yfinGens/Data1.3/{file_name}"
OUTPUT_SQL = (
    "/Users/adamtaylor/Documents/Personal/Finance/StockDB/SQL/"
    f"StkTblMerge/StkTblMerge_{file_name}.sql"
)

df = pd.read_csv(CSV_PATH)

# ===========================
# EXPECTED CSV COLUMNS (NEW)
# ===========================

expected_cols = {
    "ticker",
    "exchange",
    "company_name",
    "close",
    "daily_return",
    "rsi",
    "r1m",
    "r3m",
    "r12m",
    "ma_50",
    "ma_50_30d_ago",
    "ma_200",
    "vol_3m",
    "downside_vol_3m",
    "idiosyncratic_vol_1y",
    "sharpe_1y",
    "beta_1y",
    "market_cap",
    "sector",
    "eval_date",
    "source",
    "vol_z",
    "sharpe_1y_z",
    "downside_vol_z",
    "idiosyncratic_vol_z",
    "safety_score",
    "signal",
}

missing = expected_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing expected columns in CSV: {sorted(missing)}")

# ===========================
# SQL Formatting Helpers
# ===========================

def sql_str(val):
    if pd.isna(val):
        return "NULL"
    s = str(val).replace("'", "''")
    return f"'{s}'"

def sql_num(val):
    if pd.isna(val):
        return "NULL"
    try:
        if isinstance(val, (float, np.floating)) and not np.isfinite(val):
            return "NULL"
    except:
        pass
    return str(val)

def sql_date(val):
    if pd.isna(val):
        return "NULL"
    s = str(val)
    return f"TO_DATE('{s}', 'YYYY-MM-DD')"

def sql_sector_id(val):
    if pd.isna(val):
        return "NULL"
    name = str(val).replace("'", "''")
    return f"(SELECT sector_id FROM stock_sector WHERE sector_name = '{name}')"

# ===========================
# 1) MERGE statements for stock_sector (auto-insert)
# ===========================

sector_merges = []
unique_sectors = sorted(s for s in df["sector"].dropna().unique())

for sector_name in unique_sectors:
    stmt = f"""
MERGE INTO stock_sector t
USING (SELECT {sql_str(sector_name)} AS sector_name FROM dual) src
ON (t.sector_name = src.sector_name)
WHEN NOT MATCHED THEN
    INSERT (sector_name)
    VALUES (src.sector_name);
""".strip()
    sector_merges.append(stmt)

# ===========================
# 2) MERGE statements for stocks
# ===========================

merge_statements = []

for _, row in df.iterrows():

    stmt = f"""
    
MERGE INTO stocks s
USING (
    SELECT
        {sql_str(row['ticker'])}   AS ticker,
        {sql_str(row['exchange'])} AS exchange
    FROM dual
) src
ON (s.ticker = src.ticker AND s.exchange = src.exchange)

WHEN MATCHED THEN
    UPDATE SET
        s.company_name      = {sql_str(row['company_name'])},
        s.close_price       = {sql_num(row['close'])},
        s.daily_return      = {sql_num(row['daily_return'])},
        s.rsi               = {sql_num(row['rsi'])},
        s.return_1m         = {sql_num(row['r1m'])},
        s.return_3m         = {sql_num(row['r3m'])},
        s.return_12m        = {sql_num(row['r12m'])},
        s.ma_50             = {sql_num(row['ma_50'])},
        s.ma_50_30d_ago     = {sql_num(row['ma_50_30d_ago'])},
        s.ma_200            = {sql_num(row['ma_200'])},
        s.vol_3m            = {sql_num(row['vol_3m'])},
        s.downside_vol_3m   = {sql_num(row['downside_vol_3m'])},
        s.idiosync_vol_1y   = {sql_num(row['idiosyncratic_vol_1y'])},
        s.sharpe_1y         = {sql_num(row['sharpe_1y'])},
        s.beta_1y           = {sql_num(row['beta_1y'])},
        s.vol_z             = {sql_num(row['vol_z'])},
        s.market_cap        = {sql_num(row['market_cap'])},
        s.sector_id         = {sql_sector_id(row['sector'])},
        s.signal            = {sql_str(row['signal'])},
        s.eval_date         = {sql_date(row['eval_date'])},
        s.source            = {sql_str(row['source'])},
        s.sharpe_1y_z       = {sql_num(row['sharpe_1y_z'])},
        s.downside_vol_z    = {sql_num(row['downside_vol_z'])},
        s.idiosync_vol_z    = {sql_num(row['idiosyncratic_vol_z'])},
        s.safety_score      = {sql_num(row['safety_score'])}

WHEN NOT MATCHED THEN
    INSERT (
        ticker,
        exchange,
        company_name,
        close_price,
        daily_return,
        rsi,
        return_1m,
        return_3m,
        return_12m,
        ma_50,
        ma_50_30d_ago,
        ma_200,
        vol_3m,
        downside_vol_3m,
        idiosync_vol_1y,
        sharpe_1y,
        beta_1y,
        vol_z,
        market_cap,
        sector_id,
        signal,
        eval_date,
        source,
        sharpe_1y_z,
        downside_vol_z,
        idiosync_vol_z,
        safety_score
    ) VALUES (
        {sql_str(row['ticker'])},
        {sql_str(row['exchange'])},
        {sql_str(row['company_name'])},
        {sql_num(row['close'])},
        {sql_num(row['daily_return'])},
        {sql_num(row['rsi'])},
        {sql_num(row['r1m'])},
        {sql_num(row['r3m'])},
        {sql_num(row['r12m'])},
        {sql_num(row['ma_50'])},
        {sql_num(row['ma_50_30d_ago'])},
        {sql_num(row['ma_200'])},
        {sql_num(row['vol_3m'])},
        {sql_num(row['downside_vol_3m'])},
        {sql_num(row['idiosyncratic_vol_1y'])},
        {sql_num(row['sharpe_1y'])},
        {sql_num(row['beta_1y'])},
        {sql_num(row['vol_z'])},
        {sql_num(row['market_cap'])},
        {sql_sector_id(row['sector'])},
        {sql_str(row['signal'])},
        {sql_date(row['eval_date'])},
        {sql_str(row['source'])},
        {sql_num(row['sharpe_1y_z'])},
        {sql_num(row['downside_vol_z'])},
        {sql_num(row['idiosyncratic_vol_z'])},
        {sql_num(row['safety_score'])}
    );
""".strip()

    merge_statements.append(stmt)

# ===========================
# 3) Write output SQL file
# ===========================

with open(OUTPUT_SQL, "w") as f:
    f.write("\n\n".join(sector_merges + ["\n\n"] + merge_statements))

print("Done! Generated MERGE SQL file:")
print(OUTPUT_SQL)


Done! Generated MERGE SQL file:
/Users/adamtaylor/Documents/Personal/Finance/StockDB/SQL/StkTblMerge/StkTblMerge_stock_metrics_121625.csv.sql
